### Create dataset

In [ ]:
from pathlib import Path

from mynd.database import H5Database, create_file_database, load_file_database
from mynd.camera import ImageBundle, ImageBundleLoader

# Task specific functionality - task factories and facade
from mynd.tasks.image_export import generate_image_bundle_loaders
from mynd.tasks.image_export import CreateDatabaseTask, insert_image_bundles_into

from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


def main() -> None:
    """Main function."""

    SSD_01: Path = Path("/data/kingston_snv_01")
    SSD_02: Path = Path("/data/kingston_snv_02")

    paths: dict[str, Path] = {
        "images": SSD_02
        / Path("acfr_images_preprocessed/r23685bc_20100605_021022_images"),
        "ranges": SSD_01 / Path("acfr_stereo_ranges/r23685bc_20100605_021022_ranges"),
        "normals": SSD_01
        / Path("acfr_stereo_normals/r23685bc_20100605_021022_normals"),
        "output": SSD_01
        / Path("acfr_camera_databases/r23685bc_20100605_021022_test.h5"),
    }

    # VISITS: r23685bc_20100605_021022, r23685bc_20120530_233021, r23685bc_20140616_225022

    # Each group defines a collection of images and geometry (range and normals) captured by one sensor
    file_patterns: dict[str, str] = {
        "stereo_left": "*_LC16*",
        # "stereo_right": "*_RM16*",
    }

    config: CreateDatabaseTask.Config = CreateDatabaseTask.Config(
        output_path=paths.get("output"),
        group_patterns=file_patterns,
    )

    # --------------------------------------------------------------------------------

    bundle_loaders: list[ImageBundleLoader] = generate_image_bundle_loaders(
        image_directory=paths.get("images"),
        range_directory=paths.get("ranges"),
        normal_directory=paths.get("normals"),
        pattern=file_patterns.get("stereo_left"),
    )

    # --------------------------------------------------------------------------------

    # Create / load file database
    if not config.output_path.exists():
        result: Result[H5Database, str] = create_file_database(config.output_path)
    else:
        result: Result[H5Database, str] = load_file_database(config.output_path)

    if result.is_err():
        logger.error(result.err())
        return

    database: H5Database = result.ok()
    group: H5Database.Group = database.create_group(
        "/r23685bc_20100605_021022/left_images"
    ).unwrap()

    insertion_result: Result[None, str] = insert_image_bundles_into(
        group=group,
        bundle_loaders=bundle_loaders,
        chunk_size=100,
    )

    if insertion_result.is_err():
        logger.error(insertion_result.err())

    database.visit(logger.info)


# ---------- Run main function ----------
main()